In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from ipywidgets import GridspecLayout
from ipywidgets import Layout
import joblib

In [2]:
def one_hot_encode_variables(df, categories_dict):
    for column, categories in categories_dict.items():
        if column in df.columns:
            for category in categories:
                dummy_column = f"{column}_{category}"
                df[dummy_column] = (df[column] == category).astype(int)
            df.drop(column, axis=1, inplace=True)
    return df

def binary_encode(df, columns_to_encode):
    mapping = {'Y': 1, 'N': 0}
    df[columns_to_encode] = df[columns_to_encode].map(mapping)
    return df

def ordinal_encode(df, columns_to_encode, custom_order):
    mapping = {val: idx for idx, val in enumerate(custom_order)}
    df[columns_to_encode] = df[columns_to_encode].map(mapping)
    return df

def preprocess_data_single_entry(data_entry, categorical_dict, binary_col, ordinal_col, custom_order):
    df = pd.DataFrame(data_entry)
    df = one_hot_encode_variables(df, categorical_dict)
    df = binary_encode(df, binary_col)
    df = ordinal_encode(df, ordinal_col, custom_order)
    df.fillna(df.median(), inplace=True)
    return df


In [4]:
def f(person_age, person_income, person_home_ownership, person_emp_length, loan_intent, loan_grade,
      loan_amnt, loan_int_rate, loan_percent_income, cb_person_default_on_file, cb_person_cred_hist_length):
   
   model = joblib.load('my_random_forest.joblib')
   data_entry = {
        'person_age': [person_age],
        'person_income': [person_income],
        'person_home_ownership': [person_home_ownership],
        'person_emp_length': [person_emp_length],
        'loan_intent': [loan_intent],
        'loan_grade': [loan_grade],
        'loan_amnt': [loan_amnt],
        'loan_int_rate': [loan_int_rate],
        'loan_percent_income': [loan_percent_income],
        'cb_person_default_on_file': [cb_person_default_on_file],
        'cb_person_cred_hist_length': [cb_person_cred_hist_length]
    }
   
   categorical_dict = {
    'person_home_ownership': ['MORTGAGE',  'OTHER', 'OWN',  'RENT'],
    'loan_intent': ['DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE']
   }

   binary_col = 'cb_person_default_on_file'
   ordinal_col = 'loan_grade'
   custom_order = ['A', 'B', 'C', 'D', 'E']
   customer_data_transformed = preprocess_data_single_entry(data_entry, categorical_dict, binary_col, ordinal_col, custom_order)
   
   prediction = model.predict(customer_data_transformed)
   predicted_probability = model.predict_proba(customer_data_transformed)[:,1]

   status = "granted" if prediction[0] == 0 else "denied"
   color = "red" if prediction[0] == 1 and status == "denied" else "green"
   return f"The credit would be {status}! The risk that the loan is in default is: {predicted_probability[0]}"



# Create a function to generate HTML for a heading with custom class
def generate_heading(text, css_class=''):
    return f"<h1 class='{css_class}'>{text}</h1>"

heading_widget = widgets.HTML(value=generate_heading("Credit Helper"))

def calculate_percentage (person_income, loan_amnt):
    
    percent = loan_amnt / person_income
    round(percent, 2)
    
    return percent



# Create widgets for each criteria
person_age_widget = widgets.IntSlider(description="")
person_income_widget = widgets.FloatText(description="")
person_home_ownership_widget = widgets.Dropdown(options=['RENT', 'OWN', 'MORTGAGE', 'OTHER'], description="")
person_emp_length_widget = widgets.IntSlider(description="")
loan_intent_widget = widgets.Dropdown(options=['EDUCATION', 'DEBTCONSOLIDATION', 'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE'], description="")
loan_grade_widget = widgets.Dropdown(options=['A', 'B', 'C', 'D', 'E'], description="")
loan_amnt_widget = widgets.FloatText(description="")
loan_int_rate_widget = widgets.FloatText(description="")
loan_percent_income_widget = widgets.FloatText(description="")
cb_person_default_on_file_widget = widgets.Dropdown(options={'Yes': 'Y', 'No': 'N'}, description="")
cb_person_cred_hist_length_widget = widgets.IntSlider(description="")


# Create a button to invoke the function
invoke_button = widgets.Button(description="Calculate")
invoke_button.style.button_color = 'lightblue'
invoke_button.style.font_weight = 'bold'


def on_button_click(b):
   result_output.clear_output()
   with result_output:
       result = f(person_age_widget.value, person_income_widget.value, person_home_ownership_widget.value,
                  person_emp_length_widget.value, loan_intent_widget.value, loan_grade_widget.value,
                  loan_amnt_widget.value, loan_int_rate_widget.value,
                  calculate_percentage(person_income_widget.value, loan_amnt_widget.value), cb_person_default_on_file_widget.value,
                  cb_person_cred_hist_length_widget.value)
       print(result)
# Display widgets and their descriptions
display(heading_widget)   

label_style = {'font_weight': 'bold'}





Label_age = widgets.Label("Age")
Label_age.style = label_style

Label_Income = widgets.Label("Annual Income")
Label_Income.style = label_style

Label_Ownership = widgets.Label("Home Ownership")
Label_Ownership.style = label_style

Label_Length = widgets.Label("Employment Length (in years)")
Label_Length.style = label_style

Label_Intent = widgets.Label("Loan Intent")
Label_Intent.style = label_style

Label_Grade = widgets.Label("Loan Grade")
Label_Grade.style = label_style

Label_Amount = widgets.Label("Loan Amount")
Label_Amount.style = label_style

Label_Interest = widgets.Label("Interest Rate")
Label_Interest.style = label_style

#Label_Percent = widgets.Label("Percent of yearly income")
#Label_Percent.style = label_style

Label_Default = widgets.Label("Historical Default")
Label_Default.style = label_style

Label_History = widgets.Label("Credit History Length")
Label_History.style = label_style


#display(widgets.Label("Enter Criteria:"))
display(Label_age, person_age_widget)
display(Label_Income, person_income_widget)
display(Label_Ownership, person_home_ownership_widget)
display(Label_Length, person_emp_length_widget)
display(Label_Intent, loan_intent_widget)
display(Label_Grade, loan_grade_widget)
display(Label_Amount, loan_amnt_widget)
display(Label_Interest, loan_int_rate_widget)
#display(Label_Percent, loan_percent_income_widget)
display(Label_Default, cb_person_default_on_file_widget)
display(Label_History, cb_person_cred_hist_length_widget)


# Display the invoke button and result output
invoke_button.on_click(on_button_click)
display(invoke_button)
result_output = widgets.Output()
display(result_output)

HTML(value="<h1 class=''>Credit Helper</h1>")

Label(value='Age', style=LabelStyle(font_weight='bold'))

IntSlider(value=0)

Label(value='Annual Income', style=LabelStyle(font_weight='bold'))

FloatText(value=0.0)

...

Dropdown(options={'Yes': 'Y', 'No': 'N'}, value='Y')

Label(value='Credit History Length', style=LabelStyle(font_weight='bold'))

IntSlider(value=0)

Button(description='Calculate', style=ButtonStyle(button_color='lightblue', font_weight='bold'))

Output()